In [ ]:
import os

import gt_extras as gte
import polars as pl
import polars.selectors as cs
import polarspiper as ppp
from great_tables import (
    GT,
    exibble,
    loc,
    md,
    google_font,
    nanoplot_options,
    style,
    system_fonts,
)

from greatest_running_table.create_parquet import create_parquet

In [ ]:
parquet_file = "../data/activities.parquet"
if not os.path.exists(parquet_file):
    create_parquet(destination=parquet_file)

In [ ]:
df = (
    pl.scan_parquet(parquet_file)
    .select(cs.exclude("stream"))
    .collect()
    .with_columns(
        pl.from_epoch("time"),
        pl.from_epoch("created"),
        pl.from_epoch("edited"),
    )
    .pipe(ppp.drop_columns_that_are_all_null)
    # .with_columns(pl.col("zoneDistributionHr").fill_null(pl.lit([0] * 5)))
    # .with_columns(pl.col("zoneDistributionPace").fill_null(pl.lit([0] * 9)))
    .with_columns(
        date=pl.col("time").dt.date(),
    )
    .with_columns(sport=pl.col("sport").rank("dense"))
    .with_columns(
        duration=pl.col("duration").map_elements(
            lambda s: (f"{int(s // 3600)}h" if s >= 3600 else "")
            + (f"{int((s % 3600) // 60)}m" if s >= 60 else "")
            + (f"{int(s % 60)}s" if s < 60 or int(s % 60) > 0 else ""),
            return_dtype=pl.Utf8,
        )
    )
    .select(
        "date",
        "time",
        "sport",
        "title",
        "distance",
        "duration",
        "elevationUp",
        "elevationDown",
        "hrAvg",
        "hrMax",
        "kcal",
        # "vo2max",
        "rpe",
        "zoneDistributionPace",
        "zoneDistributionHr",
    )
    .sort("time", descending=True)
    .with_columns(pl.col("time").dt.to_string("%H:%M"))
    .head(20)
    .rename(
        mapping={
            "date": "Date",
            "time": "Time",
            "sport": "Sport",
            "title": "Title",
            "distance": "Distance",
            "duration": "Duration",
            "hrMax": "Max",
            "kcal": "kCal",
            # "vo2max": "VO2max (ml/kg/min)",
            "rpe": "RPE",
            "zoneDistributionPace": "Pace",
            "elevationUp": "Up",
            "elevationDown": "Down",
            "hrAvg": "Avg",
            "zoneDistributionHr": "HR",
        }
    )
    .to_pandas()
    .assign(
        HR=lambda d: d["HR"].apply(
            lambda x: [int(_) for _ in x] if x is not None else []
        )
    )
    .assign(
        Pace=lambda d: d["Pace"].apply(
            lambda x: [int(_) for _ in x] if x is not None else []
        )
    )
    .assign(
        # drop the timestamp from Date column
        Date=lambda d: d["Date"].astype(str).str.split(" ").str[0]
    )
)
df

In [ ]:
(
    GT(df)
    .cols_hide("Sport")
    .tab_stub(rowname_col="Time", groupname_col="Date")
    .fmt_number(columns="Distance", pattern="{x}km", decimals=1)
    .tab_spanner(label="Elevation (m)", columns=["Up", "Down"])
    .tab_spanner(label="Heart Rate (bpm)", columns=["Max", "Avg"])
    .tab_spanner(label="Zones", columns=["Pace", "HR"])
    .tab_options(
        container_width="100%",
        container_height="auto",
        row_group_background_color="WhiteSmoke",
        table_font_names=system_fonts(name="system-ui"),
    )
    .opt_vertical_padding(scale=0.1)
    .cols_width(cases={"Title": "200px"})
    .tab_style(
        style=style.fill(color="Azure"),
        locations=loc.body(
            columns="Title",
            rows=lambda d: (d["Sport"] == 4),
        ),
    )
    .tab_style(
        style=style.fill(color="Ghostwhite"),
        locations=loc.body(columns=["Up", "Down"]),
    )
    .tab_style(
        style=style.fill(color="Whitesmoke"),
        locations=loc.body(columns=["Max", "Avg"]),
    )
    .pipe(
        gte.gt_plt_donut,
        columns=["RPE"],
        size=40,
        fill="black",
        domain=[0, 20],
    )
    .pipe(
        gte.gt_plt_bar_stack,
        column="HR",
        palette="Oranges",
        width=150,
    )
    .pipe(
        gte.gt_plt_bar_stack,
        column="Pace",
        palette="Purples",
        width=150,
    )
    .tab_options(data_row_padding="0px")
    .save("table.png", scale=3)
)